In [9]:
import numpy as np
import pandas as pd
import sqlite3

#### 연습문제 1
투수들의 기록중에서 평균자책점(ERA), 투구인닝(IP), 탈삼진(SO) 기록을 찾아서  Pitcher_stats 란 테이블을 만들고, 
Eagles 테이블과 Join 하여 백넘버, 선수명, 포지션, 투구인닝, 평균자책점, 탈삼진 필드를 갖는 데이터 프레임을 만들어서 
Join 한 결과를 입력하고, 그 결과를 보이시오.

In [10]:
conn = sqlite3.connect('./test.db') 
cur = conn.cursor()

In [11]:
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Pitcher_stats \
    (id INT NOT NULL, \
     player TEXT, \
     era REAL, \
     ip REAL, \
     so INT, \
     PRIMARY KEY(id));')

In [12]:
pstats = pd.read_csv('data/pstats.csv', encoding='EUC-KR')
pstats

,player,era,ip,so
0,장민재,4.81,91.2,84
1,안영명,3.43,44.2,35
2,김범수,5.67,81.0,65


In [13]:
cur = conn.cursor()
sql = 'INSERT INTO Pitcher_stats VALUES (?, ?, ?, ?, ?);'
for i in range(3):
    cur.execute(sql, (i+1, 
                      pstats.iloc[i,0],
                      float(pstats.iloc[i,1]), 
                      float(pstats.iloc[i,2]),
                      int(pstats.iloc[i,3])))
conn.commit()

In [14]:
sql = "SELECT e.back_no, e.name, e.position, \
           p.era, p.ip, p.so \
           FROM Eagles AS e JOIN Pitcher_stats AS p \
           ON e.name like p.player;"
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();
columnPitcher = ['등번호', '선수명', '포지션', '평균자책점', '투구인닝', '탈삼진']
pitcher_df = pd.DataFrame(columns = columnPitcher)
for row in rows:
    pitcher_df = pitcher_df.append(pd.DataFrame([list(row)], columns = columnPitcher), 
                                   ignore_index=True)
pitcher_df

,등번호,선수명,포지션,평균자책점,투구인닝,탈삼진
0,17,김범수,투수,5.67,81.0,65
1,38,안영명,투수,3.43,44.2,35
2,36,장민재,투수,4.81,91.2,84


In [15]:
conn.close()

#### 연습문제 2
다음의 지시대로 DB 테이블을 만들고 이를 조회하는 프로그램을 만드시오. 
1. 국내의 대표적인 걸그룹 또는 보이그룹 5개 이상에 대하여 다음과 같은 정보를 갖는 테이블을 만드시오. \*는 Primary Key<br>
id(\*), group_name, 구성원 수, 데뷔일자, 소속사

2. 이들이 불렀던 노래 또는 다른 사람이 불렀던 노래 10곡 이상에 대하여 다음의 정보를 갖는 테이블을 만드시오.<br>
song_id(\*), song_name, 그룹 id, 발표년도, 작곡가, 도입부 가사

3. 위 두개의 테이블을 조인한 결과를 가지고 다음의 필드를 갖는 데이터 프레임을 만드시오.<br>
그룹 이름, 구성원 수, 데뷔 일자, 노래 이름, 발표 년도

In [16]:
conn = sqlite3.connect('./test.db') 
cur = conn.cursor()

In [17]:
cur.execute('CREATE TABLE IF NOT EXISTS Singers \
    (id INTEGER PRIMARY KEY, \
     name TEXT NOT NULL, \
     member_no INT, \
     debut TEXT, \
     company TEXT);')

In [18]:
singers = pd.read_csv('data/singers.csv', encoding='EUC-KR')
singers

,name,member_no,debut,company
0,방탄소년단,7,2013-06-13,빅히트 엔터
1,마마무,4,2014-06-19,RBW
2,우주소녀,13,2016-02-25,스타쉽 엔터
3,블락비,7,2011-04-15,KQ 엔터
4,엑소,9,2012-04-08,SM 엔터
5,트와이스,9,2015-10-20,JYP 엔터


In [19]:
cur = conn.cursor()
sql = 'INSERT INTO Singers VALUES (?, ?, ?, ?, ?);'
for i in range(6):
    cur.execute(sql, (i+1,
                      singers.iloc[i,0],
                      int(singers.iloc[i,1]), 
                      singers.iloc[i,2],
                      singers.iloc[i,3]))
conn.commit()

In [20]:
cur = conn.cursor()
cur.execute('SELECT * FROM Singers')
rows = cur.fetchall()
for row in rows:
    print(row)

(1, '방탄소년단', 7, '2013-06-13', '빅히트 엔터')
(2, '마마무', 4, '2014-06-19', 'RBW')
(3, '우주소녀', 13, '2016-02-25', '스타쉽 엔터')
(4, '블락비', 7, '2011-04-15', 'KQ 엔터')
(5, '엑소', 9, '2012-04-08', 'SM 엔터')
(6, '트와이스', 9, '2015-10-20', 'JYP 엔터')


In [21]:
from dateutil.parser import parse
parse('2011-04-15').date()

datetime.date(2011, 4, 15)

In [22]:
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Songs \
    (id INTEGER PRIMARY KEY, \
     name TEXT NOT NULL, \
     gid INT NOT NULL, \
     year INT, \
     composer TEXT, \
     lyrics TEXT);')

In [23]:
songs = pd.read_csv('data/songs.csv', encoding='EUC-KR')
songs

,song,gid,year,composer,lyrics
0,작은 것들을 위한 시,1,2019,Pdogg 외,모든 게 궁금해 How’s your day
1,FAKE LOVE,1,2018,Pdogg 외,널 위해서라면 난 슬퍼도 기쁜 척 할 수가 있었어
2,넌 is 뭔들,2,2016,김도훈 외,Hey 거기 미소가 예쁜 남자 바로 너
3,음오아예 (Um Oh Ah Yeh),2,2015,김도훈,oh yes 음 오 아 예 너에게 빠져들겠어 자꾸 반응하잖아
4,HER,4,2014,지코 외,Jesus 무슨 말이 필요해 모두 널 작품이라고 불러
5,으르렁,5,2013,신혁 외,나 으르렁 으르렁 으르렁 대 나 으르렁 으르렁 으르렁 대
6,MAMA,5,2012,유영진,Careless careless Shoot anonymous anonymous
7,CHEER UP,6,2016,블랙아이드필승 외,CHEER UP BABY CHEER UP BABY 좀 더 힘을 내
8,TT,6,2016,블랙아이드필승 외,이런 내 맘 모르고 너무해 너무해
9,빨간 맛,11,2017,Ludwig Lindell 외,빨간 맛 궁금해 Honey 깨물면 점점 녹아든 스트로베리 그 맛


In [24]:
cur = conn.cursor()
sql = 'INSERT INTO Songs VALUES (?, ?, ?, ?, ?, ?);'
for i in range(10):
    cur.execute(sql, (i+101,
                      songs.iloc[i,0],
                      int(songs.iloc[i,1]), 
                      int(songs.iloc[i,2]), 
                      songs.iloc[i,3],
                      songs.iloc[i,4]))
conn.commit()

In [25]:
cur = conn.cursor()
cur.execute('SELECT * FROM Songs')
rows = cur.fetchall()
for row in rows:
    print(row)

(101, '작은 것들을 위한 시', 1, 2019, 'Pdogg 외', '모든 게 궁금해 How’s your day')
(102, 'FAKE LOVE', 1, 2018, 'Pdogg 외', '널 위해서라면 난 슬퍼도 기쁜 척 할 수가 있었어')
(103, '넌 is 뭔들', 2, 2016, '김도훈 외', 'Hey 거기 미소가 예쁜 남자 바로 너')
(104, '음오아예 (Um Oh Ah Yeh)', 2, 2015, '김도훈', 'oh yes 음 오 아 예 너에게 빠져들겠어 자꾸 반응하잖아')
(105, 'HER', 4, 2014, '지코 외', 'Jesus 무슨 말이 필요해 모두 널 작품이라고 불러')
(106, '으르렁', 5, 2013, '신혁 외', '나 으르렁 으르렁 으르렁 대 나 으르렁 으르렁 으르렁 대')
(107, 'MAMA', 5, 2012, '유영진', 'Careless careless Shoot anonymous anonymous')
(108, 'CHEER UP', 6, 2016, '블랙아이드필승 외', 'CHEER UP BABY CHEER UP BABY 좀 더 힘을 내')
(109, 'TT', 6, 2016, '블랙아이드필승 외', '이런 내 맘 모르고 너무해 너무해')
(110, '빨간 맛', 11, 2017, 'Ludwig Lindell 외', '빨간 맛 궁금해 Honey 깨물면 점점 녹아든 스트로베리 그 맛')


In [26]:
# Inner Join
sql = "SELECT g.name, g.member_no, g.debut, \
           s.name, s.year \
           FROM Singers AS g INNER JOIN Songs AS s \
           ON g.id = s.gid;"
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();
columnName = ['그룹명', '멤버수', '데뷔일자', '곡명', '발표년도']
df = pd.DataFrame(columns = columnName)
for row in rows:
    df = df.append(pd.DataFrame([list(row)], columns = columnName), 
                   ignore_index=True)
df

,그룹명,멤버수,데뷔일자,곡명,발표년도
0,방탄소년단,7,2013-06-13,작은 것들을 위한 시,2019
1,방탄소년단,7,2013-06-13,FAKE LOVE,2018
2,마마무,4,2014-06-19,넌 is 뭔들,2016
3,마마무,4,2014-06-19,음오아예 (Um Oh Ah Yeh),2015
4,블락비,7,2011-04-15,HER,2014
5,엑소,9,2012-04-08,으르렁,2013
6,엑소,9,2012-04-08,MAMA,2012
7,트와이스,9,2015-10-20,CHEER UP,2016
8,트와이스,9,2015-10-20,TT,2016


In [29]:
# Left Outer Join
sql = "SELECT g.name, g.member_no, g.debut, \
           s.name, s.year \
           FROM Songs AS s LEFT OUTER JOIN Singers AS g \
           ON g.id = s.gid;"
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();
columnName = ['그룹명', '멤버수', '데뷔일자', '곡명', '발표년도']
df = pd.DataFrame(columns = columnName)
for row in rows:
    df = df.append(pd.DataFrame([list(row)], columns = columnName), 
                   ignore_index=True)
df

,그룹명,멤버수,데뷔일자,곡명,발표년도
0,방탄소년단,7,2013-06-13,작은 것들을 위한 시,2019
1,방탄소년단,7,2013-06-13,FAKE LOVE,2018
2,마마무,4,2014-06-19,넌 is 뭔들,2016
3,마마무,4,2014-06-19,음오아예 (Um Oh Ah Yeh),2015
4,블락비,7,2011-04-15,HER,2014
5,엑소,9,2012-04-08,으르렁,2013
6,엑소,9,2012-04-08,MAMA,2012
7,트와이스,9,2015-10-20,CHEER UP,2016
8,트와이스,9,2015-10-20,TT,2016
9,None,None,None,빨간 맛,2017


In [30]:
# Inner Join, 노래곡 발표년도 순
sql = "SELECT g.name, g.member_no, g.debut, \
           s.name, s.year \
           FROM Singers AS g JOIN Songs AS s \
           ON g.id = s.gid ORDER BY s.year;"
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();
columnName = ['그룹명', '멤버수', '데뷔일자', '곡명', '발표년도']
df = pd.DataFrame(columns = columnName)
for row in rows:
    df = df.append(pd.DataFrame([list(row)], columns = columnName), 
                   ignore_index=True)
df

,그룹명,멤버수,데뷔일자,곡명,발표년도
0,엑소,9,2012-04-08,MAMA,2012
1,엑소,9,2012-04-08,으르렁,2013
2,블락비,7,2011-04-15,HER,2014
3,마마무,4,2014-06-19,음오아예 (Um Oh Ah Yeh),2015
4,마마무,4,2014-06-19,넌 is 뭔들,2016
5,트와이스,9,2015-10-20,CHEER UP,2016
6,트와이스,9,2015-10-20,TT,2016
7,방탄소년단,7,2013-06-13,FAKE LOVE,2018
8,방탄소년단,7,2013-06-13,작은 것들을 위한 시,2019


In [31]:
conn.close()

#### 연습문제 3
사용자의 이름과, 비밀번호를 갖는 Users 테이블이 있다. 사용자의 이름과 비밀번호를 올바르게 입력하면 ‘성공’을 출력하고, 잘못 입력하면 ‘실패’를 출력하는 프로그램을 작성하시오.

In [32]:
conn = sqlite3.connect('./test.db') 
cur = conn.cursor()

In [33]:
cur.execute('CREATE TABLE IF NOT EXISTS Users \
    (uid TEXT PRIMARY KEY, \
     name TEXT NOT NULL, \
     password TEXT);')

In [34]:
users = pd.DataFrame({
    'uid': ['jhchoi','tkkim','kmsong','sylee','ewchung'],
    'name': ['최진행','김태균','송광민','이성열','정은원'],
    'password': ['1234','1234','1234','1234','1234']
})
users

,uid,name,password
0,jhchoi,최진행,1234
1,tkkim,김태균,1234
2,kmsong,송광민,1234
3,sylee,이성열,1234
4,ewchung,정은원,1234


In [35]:
cur = conn.cursor()
sql = 'INSERT INTO Users VALUES (?, ?, ?);'
for i in range(5):
    cur.execute(sql, (users.iloc[i,0],
                      users.iloc[i,1],
                      users.iloc[i,2]))
conn.commit()

In [36]:
cur = conn.cursor()
cur.execute("SELECT * FROM Users;")
rows = cur.fetchall()
for row in rows:
    print(row)

('jhchoi', '최진행', '1234')
('tkkim', '김태균', '1234')
('kmsong', '송광민', '1234')
('sylee', '이성열', '1234')
('ewchung', '정은원', '1234')


In [37]:
conn.close()

In [38]:
def checkUser(uid, password):
    print(uid, password)
    conn = sqlite3.connect('./test.db')
    cur = conn.cursor()
    sql = "SELECT name, password FROM Users WHERE uid like ?;"
    cur.execute(sql, (uid,))
    dbname, dbpassword = cur.fetchone()
    print(dbname, dbpassword)

    conn.close()

In [39]:
# 올바르게 입력한 경우
uid, password = input("uid password 입력> ").split()
checkUser(uid, password)

uid password 입력> tkkim 1234
tkkim 1234
김태균 1234


In [40]:
# Password가 틀린 경우
uid, password = input("uid password 입력> ").split()
checkUser(uid, password)

uid password 입력> tkkim 12
tkkim 12
김태균 1234


In [41]:
# UID를 잘못 입력한 경우
uid, password = input("uid password 입력> ").split()
checkUser(uid, password)

uid password 입력> tkkam 1234
tkkam 1234


TypeError: cannot unpack non-iterable NoneType object

- Exception을 활용하여 구현

In [42]:
CHECK_SUCCESS = 0
INVALID_UID = 1
INCORRECT_PASSWORD = 2
DATABASE_ERROR = 3

def checkUser(uid, password):
    conn = sqlite3.connect('./test.db')
    name = ''
    try:
        cur = conn.cursor()
        sql = "SELECT name, password FROM Users WHERE uid like ?;"
        cur.execute(sql, (uid,))
        dbName, dbPassword = cur.fetchone()
        if password == dbPassword:
            returnCode = CHECK_SUCCESS
            name = dbName
        else:
            returnCode = INCORRECT_PASSWORD
    except TypeError as te:
        returnCode = INVALID_UID
    except Exception as e: 
        print('예외가 발생했습니다.', e)
        returnCode = DATABASE_ERROR
    finally:
        conn.close()
        return returnCode, name

In [45]:
uid, password = input("uid password 입력> ").split()
result, name = checkUser(uid, password)
if result == CHECK_SUCCESS:
    print('로그인 성공, 사용자 이름 =', name)
elif result == INVALID_UID:
    print('실패 - 잘못된 User ID')
elif result == INCORRECT_PASSWORD:
    print('실패 - 패스워드 불일치')
else:
    print('실패 - 데이터베이스 에러')

uid password 입력> tkkam 1234
실패 - 잘못된 User ID


#### 패스워드 암호화를 적용한 경우
- pip install bcrypt

In [46]:
import bcrypt

In [47]:
salt=bcrypt.gensalt()
password="1234"
hashed=bcrypt.hashpw(password.encode('utf-8'), salt)
hashed

b'$2b$12$ySVzsnOVksNcNKn1dkwSMe8.QrKXSLScglXbGkii5.l71So4Cxjw6'

In [48]:
if hashed == bcrypt.hashpw('1234'.encode('utf-8'), hashed):
    print('Correct Password')

Correct Password


In [49]:
print(bcrypt.checkpw(bytes('1234', encoding="utf-8"), hashed))

True


In [50]:
print(bcrypt.checkpw(bytes('12534', encoding="utf-8"), hashed))

False


In [51]:
conn = sqlite3.connect('./test.db') 
cur = conn.cursor()
sql = 'INSERT INTO Users VALUES (?, ?, ?);'
cur.execute(sql, ('sunlee', '이태양', hashed))
conn.commit()

In [60]:
# 암호화된 패스워드 비교
def checkHashedUser(uid, password):
    print(uid, password)
    conn = sqlite3.connect('./test.db')
    name = ''
    try:
        cur = conn.cursor()
        sql = "SELECT name, password FROM Users WHERE uid like ?;"
        cur.execute(sql, (uid,))
        dbName, dbPassword = cur.fetchone()
        if bcrypt.checkpw(bytes(password, encoding='utf-8'), dbPassword):
            returnCode = CHECK_SUCCESS
            name = dbName
        else:
            returnCode = INCORRECT_PASSWORD
    except TypeError as te:
        returnCode = INVALID_UID
    except Exception as e: 
        print('예외가 발생했습니다.', e)
        returnCode = DATABASE_ERROR
    finally:
        conn.close()
        return returnCode, name

In [64]:
cur = conn.cursor()
sql = "UPDATE Users set password=? WHERE uid not like 'sunlee'"
cur.execute(sql, (hashed,))
conn.commit()

In [66]:
uid, password = input("uid password 입력> ").split()
result, name = checkHashedUser(uid, password)
if result == CHECK_SUCCESS:
    print('로그인 성공, 사용자 이름 =', name)
elif result == INVALID_UID:
    print('실패 - 잘못된 User ID')
elif result == INCORRECT_PASSWORD:
    print('실패 - 패스워드 불일치')
else:
    print('실패 - 데이터베이스 에러')

uid password 입력> sunlee 12345
sunlee 12345
실패 - 패스워드 불일치
